In [72]:
import litellm
import time
import sys, os

if not os.environ["OPENAI_API_KEY"]: 
    os.environ["OPENAI_API_KEY"]= '<REDACTED>'

In [73]:
#MODEL = "gpt-4o"
#MODEL = "gpt-4o-mini"
MODEL = "ft:gpt-4o-mini-2024-07-18:personal:20qs-tuned:9ujvexa0"
EVALUATOR_MODEL="gpt-4o-mini"
# MODEL = "gpt-4o"
#MODEL = "claude-3-5-sonnet-20240620"
litellm.modify_params = True

In [74]:
def completion(model, messages):
    result = litellm.completion(model, messages)
    if "claude" in model:
        time.sleep(60/50) # Adjust based on rate limit (https://console.anthropic.com/settings/limits)
    return result

In [4]:
class DecisionNode:
    def __init__(self, query = None, yes_branch=None, no_branch=None):
        self.query = query
        self.yes_branch = yes_branch
        self.no_branch = no_branch
        
    def print_tree(self, level=0, prefix=""):
        if self.query:
            print(f"{prefix}-- {self.query}")
            if self.yes_branch:
                self.yes_branch.print_tree(level + 1, prefix + "  |")
            if self.no_branch:
                self.no_branch.print_tree(level + 1, prefix + "   ")
        else:
            print(f"{prefix}-- *")
            
    def write_tree_to_file(self, filename):
        with open(filename, 'w') as f:
            self._write_tree(f)

    def _write_tree(self, file, level=0, prefix=""):
        if self.query:
            file.write(f"{prefix}-- {self.query}\n")
            if self.yes_branch:
                self.yes_branch._write_tree(file, level + 1, prefix + "  |")
            if self.no_branch:
                self.no_branch._write_tree(file, level + 1, prefix + "   ")
        else:
            file.write(f"{prefix}-- *\n")

    @staticmethod
    def read_tree_from_file(filename):
        with open(filename, 'r') as f:
            lines = f.readlines()
        return DecisionNode._read_tree(lines, 0, "")

    @staticmethod
    def _read_tree(lines, indent_level, current_prefix):
        if not lines:
            return None

        line = lines.pop(0).rstrip()
        expected_prefix = current_prefix + "-- "

        if not line.startswith(expected_prefix):
            return None

        content = line[len(expected_prefix):]
        if content == "*":
            return DecisionNode()

        yes_branch = DecisionNode._read_tree(lines, indent_level + 1, current_prefix + "  |")
        no_branch = DecisionNode._read_tree(lines, indent_level + 1, current_prefix + "   ")

        return DecisionNode(content, yes_branch, no_branch)

# Creating the tree

In [5]:
MC_tree = DecisionNode.read_tree_from_file('20qs-data/decision_trees/decision_tree_v2.txt')
MC_tree.print_tree()

-- food, beverage, cooking
  |-- food
  |  |-- agriculture
  |  |  |-- *
  |  |   -- prepared food
  |  |     |-- *
  |  |      -- *
  |   -- agriculture
  |     |-- safety
  |     |  |-- *
  |     |   -- *
  |      -- handheld
  |        |-- beverage
  |        |  |-- *
  |        |   -- *
  |         -- *
   -- industry, manufacturing
     |-- safety
     |  |-- transportation, vehicles
     |  |  |-- *
     |  |   -- architecture, construction
     |   -- electronics
     |     |-- *
     |      -- natural material, resource
     |        |-- *
     |         -- *
      -- sports, entertainment
        |-- handheld
        |  |-- electronics
        |  |  |-- *
        |  |   -- arts, media
        |  |     |-- *
        |  |      -- *
        |   -- *
         -- clothing, accessories, beauty
           |-- *
            -- animal
              |-- *
               -- handheld
                 |-- electronics
                 |  |-- *
                 |   -- safety
                

In [82]:
import pickle

with open('20qs-data/decision_trees/gpt_decision_tree_v2.p', 'rb') as f:
    gpt_tree_v2 = pickle.load(f)
    
with open('20qs-data/decision_trees/gpt_decision_tree_v3.p', 'rb') as f:
    gpt_tree_v3 = pickle.load(f)
    
with open('20qs-data/decision_trees/gpt_decision_tree_v3.p', 'rb') as f:
    gpt_tree_v4 = pickle.load(f)
    
big_tree = DecisionNode.read_tree_from_file('20qs-data/decision_trees/big_tree.txt')
    
big_tree.print_tree()

-- Is it related to food, beverages or cooking?
  |-- Is it an edible item?
  |  |-- Is it a plant or derived from a plant?
  |  |  |-- Is it commonly found in Western cuisine?
  |  |  |  |-- Is it typically cooked before consumption?
  |  |  |  |  |-- Is it typically green in color when fresh?
  |  |  |  |  |  |-- Is it a type of leafy green?
  |  |  |  |  |  |  |-- Does it have a strong or pungent flavor?
  |  |  |  |  |  |  |  |-- Is it commonly used as an herb or garnish rather than as a primary ingredient?
  |  |  |  |  |  |  |  |  |-- *
  |  |  |  |  |  |  |  |   -- *
  |  |  |  |  |  |  |   -- Is it considered a bitter green?
  |  |  |  |  |  |  |     |-- *
  |  |  |  |  |  |  |      -- *
  |  |  |  |  |  |   -- Is it a vegetable?
  |  |  |  |  |  |     |-- Is it a fruit vegetable?
  |  |  |  |  |  |     |  |-- *
  |  |  |  |  |  |     |   -- *
  |  |  |  |  |  |      -- Is it typically used as a seasoning or garnish?
  |  |  |  |  |  |        |-- *
  |  |  |  |  |  |         --

In [6]:
query_map = {
    'food': "Is it a food?",
    'beverage': "Is it a beverage?",
    'living': "Is it a living thing?",
    'plant': "Is it a plant?",
    'animal': 'Is it an animal?',
    'electronics': "Is it related to electronics or technology?",
    'furniture': 'Is it furniture?',
    'transportation, vehicles': 'Is it related to transportation or vehicles?',
    'man-made': "Is it a man-made thing?",
    'architecture, construction': 'Is it related to architectural structures or construction?',
    'natural material, resource': 'Is it a natural material or resource?',
    'natural phenomenon': 'Is it a natural phenomenon?',
    'industry, manufacturing': "Is it related to industrial production or manufacturing?",
    'food, beverage, cooking': 'Is it related to food, beverages or cooking?',
    'handheld': 'Is it something a person can hold in their hand?',
    'agriculture': "Is it related to agriculture?",
    'indoors': 'Is it something that can be found indoors?',
    'arts, media': "Is it broadly related to the arts or media?",
    'safety': "Is it related to safety or safety equipment?",
    'medicine': "Is it broadly related to medicine or healthcare?",
    'clothing, accessories, beauty': "Is it related to clothing, accessories or beauty products?",
    'sports, entertainment': "Is it broadly related to entertainment or sports?",
    'prepared food': "Is it a prepared food or dish?"
}


In [86]:
def game_to_string(questions, answers, guesses):
    game_text = ""
    turn = len(questions)
    for i in range(turn):
        game_text += f"Question {i+1}: {questions[i]}\n"
        game_text += f"Answer {i+1}: {answers[i]}\n"  
        if i < turn-1: game_text += f"Guess {i+1}: {guesses[i]}\n"
    return game_text

def construct_messages(game, role, keyword=None, guesses=[],  questions = [], answers = []):
    if role == "questioner":
        prompt = {
            "role": "system",
            "content": (
                "You are an AI assistant playing the 20 Questions game. In this game the Answerer is given a secret keyword. "
                "The Questioner then asks yes-or-no questions regarding the keyword, and the Answerer answers them accurately. "
                "Then the Guesser tries to guess the keyword based on the questions and answers in the game. The keyword is a specific thing, NOT a place and NOT a person.\n\n "
                "You are participating in a new game of 20 Questions. Your role is to be the Questioner. You will ask successive yes-or-no questions to determine the keyword. "
                "You have a limited number of questions to ask, so choose a question that will eliminate half of the possible keywords to maximize efficiency. "
                "Avoid asking questions that are too specific too early on. Ask questions that are as broad as possible while still eliminating half of the remaining possibilities. "
                "DO NOT ask if the keyword is a specific thing, rather ask something about the keyword.\n"
                "Example 1: DO NOT ASK: 'Is the keyword car?', INSTEAD ASK: 'Is it a specific type of car, such as a sedan?\n"
                "Example 2: DO NOT ASK: 'Is the keyword cow?', INSTEAD ASK: 'Is it a specific type of cow?\n"
                "Example 3: DO NOT ASK: 'Is the keyword bottle?', INSTEAD ASK: 'Is it a bottle made of a specific material?\n"
                "Example 4: DO NOT ASK: 'Is the keyword lamp?', INSTEAD ASK: 'Is it a type of lamp?\n"
                "Do NOT assume the game has ended, the game will determine when to stop. Do not output any text other than the question."
            ),
        }
        messages = [prompt]
        for message in game:
            if message["role"] == "questioner":
                messages.append({
                    "role": "assistant",
                    "content": message["content"]
                })
            else:
                messages.append({
                    "role": "user",
                    "content": message["content"] + "\nAsk your next question. Remember to NOT ask if the keyword is a specific thing, but rather ask something about the keyword."
                })
    elif role == "answerer":
        prompt = {
            "role": "system",
            "content": (
                f"You are an AI assistant playing the 20 Questions game. In this game the Answerer is given a secret keyword. "
                "The Questioner then asks yes-or-no questions regarding the keyword, and the Answerer answers them accurately. "
                "Then the Guesser tries to guess the keyword based on the questions and answers in the game. The keyword is a specific thing, NOT a place and NOT a person. "
                "You are participating in a new game of 20 Questions. Your role is to be the Answerer. "
                f"The keyword is {keyword}. Answer only Yes or No based on the keyword. Do not output any other text."
            ),
        }
        messages = [prompt]
        for message in game:
            if message["role"] == "questioner":
                messages.append({
                    "role": "user",
                    "content": f"Answer the following question about the keyword: '{keyword}'. " + message["content"]
                })
            else:
                messages.append({
                    "role": "assistant",
                    "content": message["content"]
                })
    elif role == "guesser":
        prompt = {
            "role": "system",
            "content": (
                "You are an AI assistant playing the 20 Questions game. In this game the Answerer is given a secret keyword. "
                "The Questioner then asks yes-or-no questions regarding the keyword, and the Answerer answers them accurately. "
                "Then the Guesser tries to guess the keyword based on the questions and answers in the game. The keyword is a specific thing, NOT a place and NOT a person. "
                "You are participating in a new game of 20 Questions. Your role is to be the Guesser. Based on the given questions and answers, guess the keyword at this point. "
                "Even if information is limited, guess a keyword. Do not ask a question, just state the guessed keyword with no other text except the keyword itself. "
                "DO NOT output any other text other than the guessed keyword. DO NOT refuse to guess. DO NOT REPEAT A PREVIOUS GUESS."
            ),
        }
        messages = [prompt, {
            "role": "user",
            "content": "Game so far:\n" + game_to_string(questions, answers, guesses) +"\nBased on all questions and answers so far, enter your guess. Do not repeat a guess.\nGuess:"
        }]
    elif role == "evaluator":
        prompt = {
            "role": "system",
            "content": (
                "You are an evaluator for the game 20 Questions. Given a keyword and a guess, return True if the guess was correct and False otherwise. "
                "The guess should refer to the same thing as the keyword but the guess should NOT be more vague or general than the keyword. DO NOT output anything except 'True' or 'False'."
                "Here are some guiding examples: \n\n"
                "Example 1:\nKeyword: USA\n Guess: United States\nEvaluation: True\n\n"
                "Example 2:\nKeyword: Peking duck\n Guess: duck\nEvaluation: False\n\n"
                "Example 3:\nKeyword: iPad\n Guess: tablet\nEvaluation: False\n\n"
                "Example 4:\nKeyword: pretoria south africa\n Guess: Pretoria\nEvaluation: True\n\n"
                "Example 5:\nKeyword: Bed and Breakfast\n Guess: BnB\nEvaluation: True\n\n"
                "Example 6:\nKeyword: Diet coke\n Guess: coca-cola\nEvaluation: False\n\n"
            ),
        }
        messages = [prompt, {
            "role": "user",
            "content": f"Keyword: {keyword}\nGuess: {guesses[-1]}\nEvaluation: "
        }]
    return messages


In [9]:
construct_messages([{"role": "questioner", "content": "Is it a place?"}], "answerer", "chair")

[{'role': 'system',
  'content': 'You are an AI assistant playing the 20 Questions game. In this game the Answerer is given a secret keyword. The Questioner then asks yes-or-no questions regarding the keyword, and the Answerer answers them accurately. Then the Guesser tries to guess the keyword based on the questions and answers in the game. The keyword is a specific thing, NOT a place and NOT a person. You are participating in a new game of 20 Questions. Your role is to be the Answerer. The keyword is chair. Answer only Yes or No based on the keyword. Do not output any other text.'},
 {'role': 'user',
  'content': "Answer the following question about the keyword: 'chair'. Is it a place?"}]

## Game Simulator

In [85]:
def simulate_game(keyword, tree, print_game = True):
    if print_game: print(f"USING MODEL: {MODEL}")
    if print_game: print(f"KEYWORD: {keyword}")
    if print_game: print()
    
    game = []
    questions = [step["content"] for step in game if step["role"] == "questioner"]
    answers = [step["content"] for step in game if step["role"] == "answerer"]
    guesses = []
    game_won = False
    decision_node = tree
    
    for turn in range(20):
        
        if decision_node and decision_node.query:
            if "?" in decision_node.query:
                question = decision_node.query
            else:
                question = query_map[decision_node.query]
        else:
            question = completion(model=MODEL, messages=construct_messages(game, "questioner", keyword)).choices[0].message.content
    
        game.append({
            "role": "questioner",
            "content": question
        })

        if print_game: print(f"questioner: {question}")

        if "OVER" in question:
            break
        
        questions.append(question)

        answer = completion(model=MODEL, messages=construct_messages(game, "answerer", keyword)).choices[0].message.content
    
        game.append({
            "role": "answerer",
            "content": answer
        })

        answers.append(answer)
        if print_game: print(f"answerer: {answer}")

        if decision_node and decision_node.yes_branch != None and "yes" in answer.lower():
            decision_node = decision_node.yes_branch
        elif decision_node and decision_node.no_branch != None and "no" in answer.lower():
            decision_node = decision_node.no_branch
        else:
            decision_node = None
            
        guess = completion(model=MODEL, messages=construct_messages(game, "guesser", keyword, guesses, questions, answers)).choices[0].message.content

        guesses.append(guess)
        if print_game: print(f"guesser: {guess}")

        evaluation = completion(model=EVALUATOR_MODEL, messages=construct_messages(game, "evaluator", keyword, guesses)).choices[0].message.content
        if print_game: print(f"evaluator: {evaluation}")
        if "true" in evaluation.lower(): 
            game_won = True
            break

        if print_game: print()

    return {
        "keyword": keyword,
        "questions": questions,
        "answers": answers,
        "guesses": guesses,
        "win" : game_won,
    }

In [84]:
simulate_game("barbell", big_tree)

USING MODEL: ft:gpt-4o-mini-2024-07-18:personal:20qs-tuned:9ujvexa0
KEYWORD: barbell

!
questioner: Is it related to food, beverages or cooking?
answerer: no
guesser: Smartphone
evaluator: False

!
questioner: Is it typically found inside a home?
answerer: yes
guesser: Television
evaluator: False

!
questioner: Is it commonly found in a bathroom?
answerer: no
guesser: Lamp
evaluator: False

!
questioner: Is it typically found in a living room?
answerer: no
guesser: Refrigerator
evaluator: False

!
questioner: Is it primarily used for storage or organization?
answerer: no
guesser: Computer
evaluator: False

!
questioner: Is it a type of tool?
answerer: yes
guesser: Hammer
evaluator: False

!
questioner: Is it commonly associated with painting or decorating tasks?
answerer: no
guesser: Screwdriver
evaluator: False

!
questioner: Is it generally handheld?
answerer: yes
guesser: Electric drill
evaluator: False

questioner: Is it used in the kitchen?
answerer: no
guesser: Flashlight
evaluat

{'keyword': 'barbell',
 'questions': ['Is it related to food, beverages or cooking?',
  'Is it typically found inside a home?',
  'Is it commonly found in a bathroom?',
  'Is it typically found in a living room?',
  'Is it primarily used for storage or organization?',
  'Is it a type of tool?',
  'Is it commonly associated with painting or decorating tasks?',
  'Is it generally handheld?',
  'Is it used in the kitchen?',
  'Is it commonly used for repairs or maintenance around the house?',
  'Is it used for cleaning purposes?',
  'Is it an electrical tool or device?',
  'Is it related to gardening or outdoor activities?',
  'Is it used for crafting or DIY projects?',
  'Is it associated with personal grooming or care?',
  'Is it a type of measuring tool?',
  'Is it used for office or school work?',
  'Is it used for sewing or mending clothes?',
  'Is it used for personal fitness or exercise?',
  'Is it primarily used for strength training?'],
 'answers': ['no',
  'yes',
  'no',
  'no',

## Test trees:

In [77]:
first_100_test_words = ['Dandelion', 'weasel', 'pot holder', 'casket', 'push pin', 'Protein bar', 'junction box', 'bonsai tree', 'Nail Polish Remover', 'livestock trailer', 'amazon echo', 'wire brush', 'rocketship', 'Paper towels', 'dresser', 'lemming', 'pear tree', 'conference table', 'extension ladder', 'vortex mixer', 'Wine Aerator', 'Golf Cart', 'Tweezers', 'catalytic converter', 'Ointment', 'Frappuccino', 'aluminum foil', 'headboard', 'butter knife', 'chimney', 'sewing needle', 'angelfish', 'bird netting', 'level sensor', 'cane', 'palmetto tree', 'coal crusher', 'Sprinkler', 'king snake', 'plant saucer', 'beer coaster', 'espresso machine', 'desk lamp', 'Wine Tasting Kit', 'fossil', 'oatmeal cookie', 'rubber', 'Pallet Jack', 'butter knife', 'Sticker', 'lace dress', 'wind vane', 'Mouse', 'fudge brownie', 'Sippy cup', 'loose change', 'baggage conveyor', 'zinc', 'Screwdriver', 'binder clip', 'Matches', 'pumpkin', 'medication dispenser', 'electronic piano', 'Dish soap', 'tea cup', 'Air Conditioner', 'Reptile', 'Pressure gauge', 'Toiletries bag', 'safety glasses', 'fishing net', 'Swing set', 'Grapefruit', 'neon light', 'Stain Remover', 'Screwdriver', 'game board', 'climbing rope', 'Ventilation System', 'Pumpkin pie', 'Olive Oil', 'infrared sensor', 'petunia', 'anchovy', 'user manual', 'upright piano', 'lifting crane', 'reading pillow', 'Spatula', 'Stain Remover', 'Komodo dragon', 'Floor jacks', 'sweetgum tree', 'grape juice', 'ultrasound', 'plastic baggie', 'Soldering iron', 'Reptile', 'caribou']

In [78]:
from tqdm.notebook import tqdm
import numpy as np

trees_map = {'gpt_tree_v2': gpt_tree_v2, 'gpt_tree_v3': gpt_tree_v3, 'gpt_tree_v4': gpt_tree_v4, 'MC_tree': MC_tree}
test_results = {'gpt_tree_v2': {}, 'gpt_tree_v3': {}, 'gpt_tree_v4': {}, 'MC_tree': {}}
    

trees:   0%|          | 0/4 [00:00<?, ?it/s]

keywords:   0%|          | 0/100 [00:00<?, ?it/s]

keywords:   0%|          | 0/100 [00:00<?, ?it/s]

keywords:   0%|          | 0/100 [00:00<?, ?it/s]

keywords:   0%|          | 0/100 [00:00<?, ?it/s]

Tree: gpt_tree_v2
Number of keywords guessed correctly: 19
Average length of games won: 19
Keywords guessed correctly:
Ointment
butter knife
oatmeal cookie
Mouse
fudge brownie
Screwdriver
Dish soap
Air Conditioner
safety glasses
fishing net
neon light
game board
climbing rope
petunia
lifting crane
Spatula
Komodo dragon
grape juice
plastic baggie

Tree: gpt_tree_v3
Number of keywords guessed correctly: 30
Average length of games won: 30
Keywords guessed correctly:
weasel
Protein bar
amazon echo
dresser
Golf Cart
Tweezers
cane
king snake
espresso machine
fossil
oatmeal cookie
Sticker
Mouse
fudge brownie
Screwdriver
binder clip
electronic piano
Dish soap
Air Conditioner
Reptile
Swing set
Grapefruit
game board
Pumpkin pie
petunia
anchovy
lifting crane
Spatula
grape juice
caribou

Tree: gpt_tree_v4
Number of keywords guessed correctly: 25
Average length of games won: 25
Keywords guessed correctly:
weasel
Protein bar
amazon echo
Tweezers
palmetto tree
Sprinkler
king snake
desk lamp
oatmeal c

In [83]:
import random

test_words = random.choices([keyword for keyword in keyword_things if keyword not in test_results['gpt_tree_v2']], k = 150)

In [86]:
print(test_words)

['groundhog', 'Utility Cart', 'Pamphlet', 'cockroach', 'bank statement', 'vegetarian chili', 'Luggage', 'oxygen', 'planter', 'butterfly net', 'parrot', 'smoke alarm', 'hand drill', 'peony', 'fire helmet', 'cupcake', 'hot tea', 'Hairbrush', 'Lathe', 'meatloaf', 'dust jacket', 'medical chart', 'bald eagle', 'Rat', 'pipe', 'hazelnut tree', 'pineapple', 'cable tie', 'steam iron', 'Ultraviolet Lamp', 'Phone charger', 'hair elastic', 'wine decanter', 'Umbrella stand', 'Ear protection', 'pillow sham', 'volleyball', 'metal file', 'earring', 'charger', 'honeydew', 'relief valve', 'Bulb', 'teddy bear', 'silicon', 'Ironing Board', 'chair cushion', 'microphone stand', 'hamster', 'wire hanger', 'jacket', 'Mushroom', 'elastic bandage', 'Muffin Tin', 'doughnut', 'postage stamp', 'Fungi', 'picture frame', 'vest', 'cooler', 'perfume bottle', 'foil paper', 'Ocarina', 'Index Card', 'steak', 'Ear protection', 'Radio scanner', 'chicken wire', 'Floor Mats', 'rice krispie', 'moving walkway', 'Disinfectant', 

In [88]:
for tree in tqdm(trees_map, desc = 'trees'):
    for keyword in tqdm(test_words, leave = False, desc = 'keywords'):
        game_result = simulate_game(keyword, trees_map[tree], print_game = False)
        test_results[tree][keyword] = game_result
        
for tree in test_results:
    keywords_guessed_correctly = [keyword for keyword in test_results[tree] if test_results[tree][keyword]["win"] == True]
    print(f"Tree: {tree}")
    print(f"Number of keywords guessed correctly: {len(keywords_guessed_correctly)}")
    keywords_print = "\n".join(keywords_guessed_correctly)
    avg_len_games_won = np.mean([len(test_results[tree][keyword]['questions']) for keyword in keywords_guessed_correctly])
    print(f"Average length of games won: {avg_len_games_won}")
    print(f"Keywords guessed correctly:\n{keywords_print}\n")

with open("20qs-data/final_games/results_by_tree.jsonl", "w") as f:
    json.dump(test_results, f)

trees:   0%|          | 0/4 [00:00<?, ?it/s]

keywords:   0%|          | 0/150 [00:00<?, ?it/s]

keywords:   0%|          | 0/150 [00:00<?, ?it/s]

keywords:   0%|          | 0/150 [00:00<?, ?it/s]

keywords:   0%|          | 0/150 [00:00<?, ?it/s]

Tree: gpt_tree_v2
Number of keywords guessed correctly: 52
Average length of games won: 14.076923076923077
Keywords guessed correctly:
Ointment
butter knife
oatmeal cookie
Mouse
fudge brownie
Screwdriver
Dish soap
Air Conditioner
safety glasses
fishing net
neon light
game board
climbing rope
petunia
lifting crane
Spatula
Komodo dragon
grape juice
plastic baggie
Luggage
cupcake
hot tea
Lathe
Rat
volleyball
charger
honeydew
Bulb
jacket
Mushroom
doughnut
Fungi
picture frame
cooler
steak
Disinfectant
Hazelnut
Agave
Sleeping bag
Milling machine
garden shovel
Gym Mats
mop
coffee machine
Stair Stepper
lime
impact crusher
strainer
Stove
Oxygen tank
beer glass
Pants

Tree: gpt_tree_v3
Number of keywords guessed correctly: 67
Average length of games won: 14.656716417910447
Keywords guessed correctly:
weasel
Protein bar
amazon echo
dresser
Golf Cart
Tweezers
cane
king snake
espresso machine
fossil
oatmeal cookie
Sticker
Mouse
fudge brownie
Screwdriver
binder clip
electronic piano
Dish soap
Air Co

## Generate all the games

In [92]:
import json

def read_jsonl_and_transform(file_path):
    result_dict = {}

    with open(file_path, 'r') as file:
        for line in file:
            record = json.loads(line)
            keyword = record.pop('keyword')
            result_dict[keyword] = record

    return result_dict

def read_json_to_dict(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

keyword_labels, results = read_jsonl_and_transform('20qs-data/keywords_data/labeled_keywords_v2.jsonl'), read_json_to_dict('20qs-data/keywords_data/labeling_results_v2.jsonl')

def read_file_to_list(file_path):
    with open(file_path, 'r') as file:
        lines = file.read().splitlines()
    return lines

keywords_list = read_file_to_list('20qs-data/keywords_data/keywords.txt')
similar_keywords_list = read_file_to_list('20qs-data/keywords_data/similar_keywords.txt')
twice_removed_keywords_list = read_file_to_list('20qs-data/keywords_data/twice_removed_more_similar_keywords.txt')

keywords_list = keywords_list[1:]
print(f"Number of keywords in data: {len(keywords_list)}")

keyword_places = [keyword for keyword in keywords_list if keyword_labels[keyword]['place'] == 'yes']
keyword_things = [keyword for keyword in keywords_list if keyword_labels[keyword]['place'] == 'no']

print(f"Number of keywords labeled as things: {len(keyword_things)}.")

Number of keywords in data: 2046
Number of keywords labeled as things: 1347.


Read current games:

In [61]:
import json

def read_jsonl_and_transform(file_path):
    result_dict = {}

    with open(file_path, 'r') as file:
        for line in file:
            if line.strip():  # Skip empty lines
                try:
                    record = json.loads(line)
                    keyword = record.pop('keyword')
                    result_dict[keyword] = record
                except json.JSONDecodeError as e:
                    print(f"Error decoding JSON for line: {line}")
                    print(f"Error message: {e}")

    return result_dict

# Path to the uploaded file
#file_path = '20qs-data/real_keyword_games_extended.jsonl'

# Read and transform the JSONL file
#result_dict = read_jsonl_and_transform(file_path)

# Get the current keywords with games
current_keywords_with_games = read_jsonl_and_transform('20qs-data/final_games/530-games-for-training.jsonl')

In [88]:
game_results = []

In [93]:
import random
from tqdm.notebook import tqdm

dict_keywords = {d['keyword'] for d in game_results}

keywords_left = [keyword for keyword in keyword_things if (keyword not in dict_keywords) and (keyword not in current_keywords_with_games)]
similar_keywords_left = [keyword for keyword in similar_keywords_list if (keyword not in dict_keywords) and (keyword not in current_keywords_with_games)]

print(f"Keywords left: {len(keywords_left)}\n")
print(f"Similar Keywords left: {len(similar_keywords_left)}\n")

generated_keywords = random.choices(keywords_left, k=100)
generated_similar_keywords = random.choices(similar_keywords_left, k=100)
generated_keywords_combined = list(set(generated_keywords + generated_similar_keywords))

for generated_keyword in tqdm(twice_removed_keywords_list, desc = 'games generated'):
    print("Generated keyword:", generated_keyword)
    game_result = simulate_game(generated_keyword, MC_tree)
    game_results.append(game_result)

Keywords left: 1074

Similar Keywords left: 984



games generated:   0%|          | 0/348 [00:00<?, ?it/s]

Generated keyword: Toggle bolt
USING MODEL: ft:gpt-4o-mini-2024-07-18:personal:20qs-tuned:9ujvexa0
KEYWORD: Toggle bolt

questioner: Is it related to food, beverages or cooking?
answerer: no
guesser: Car
evaluator: False

questioner: Is it related to industrial production or manufacturing?
answerer: yes
guesser: Robot
evaluator: False

questioner: Is it related to safety or safety equipment?
answerer: no
guesser: Factory
evaluator: False

questioner: Is it related to electronics or technology?
answerer: no
guesser: Machine
evaluator: False

questioner: Is it a natural material or resource?
answerer: no
guesser: Tool
evaluator: False

questioner: Is it commonly used in construction or building?
answerer: yes
guesser: Concrete
evaluator: False

questioner: Is it typically used as a structural component?
answerer: no
guesser: Insulation
evaluator: False

questioner: Is it primarily used for finishing or interior work?
answerer: yes
guesser: Paint
evaluator: False

questioner: Is it common

KeyboardInterrupt: 

In [95]:
new_games_won_words = [game for game in game_results if game['win'] == True]
print(len(new_games_won_words))

155


In [97]:
print(keyword_things)

['acacia', 'accent chair', 'acorn', 'adhd medication', 'adjustable bench', 'Advertisement', 'aerator', 'Agave', 'agave plant', 'Air compressor', 'Air Conditioner', 'Air filter', 'air handler', 'air plant', 'air tank', 'Air vent', 'airport lighting', 'Alarm system', 'alfalfa', 'almond milk', 'aluminum foil', 'amazon echo', 'analog stick', 'Analogy', 'anchovy', 'Anemone', 'Anesthesia', 'angelfish', 'ankle bracelet', 'antidepressant', 'antiseptic wipe', 'aperol spritz', 'apple', 'Apple pie', 'applesauce', 'apricot tree', 'Aprons', 'aquarium salt', 'archery target', 'arrival board', 'aspirin', 'Atmosphere', 'attic ladder', 'audio guide', 'autograph', 'automatic door', 'backgammon board', 'Backrest', 'bacon', 'Bacteria', 'baggage conveyor', 'Baguette', 'balance beam', 'bald eagle', 'ballpoint pen', 'bamboo', 'bamboo leaf', 'banana', 'bandana', 'bank statement', 'bar towel', 'barbell collar', 'Barber Chair', 'barcode', 'Bat', 'Bath Mat', 'battery', 'battery bank', 'beach chair', 'beach umbre

In [69]:
keywords_left = [keyword for keyword in keyword_things if (keyword not in dict_keywords) and (keyword not in current_keywords_with_games)]
similar_keywords_left = [keyword for keyword in similar_keywords_list if (keyword not in dict_keywords) and (keyword not in current_keywords_with_games)]

print(f"Keywords left: {len(keywords_left)}\n")
print(f"Similar Keywords left: {len(similar_keywords_left)}\n")

generated_keywords = random.choices(keywords_left, k=100)
generated_similar_keywords = random.choices(similar_keywords_left, k=100)
generated_keywords_combined = list(set(generated_keywords + generated_similar_keywords))

Keywords left: 1006

Similar Keywords left: 925



In [70]:
print(generated_keywords_combined)

['projector', 'Yoga Blocks', 'Rearview mirror', 'bamboo', 'Hibiscus', 'hay bale', 'Hedgehog', 'bee', 'mulch', 'Necklace', 'Sparkling water', 'turmeric', 'Toothbrush', 'water tank', 'Device', 'snowmobile', 'Cedar', 'Bluetooth headphones', 'Leather notebook', 'footbridge', 'Disinfectant wipes', 'ignition system', 'Vending Machine', 'maidenhair fern', 'Ray-Ban sunglasses', 'Plastic Gloves', 'Carrot cake', 'grape', 'humus', 'guppy', 'Circuit breaker', 'Moscow Mule', 'Vinaigrette', 'Champagne flutes', 'sword fern', 'rocking chair', 'serving bowl', 'tire gauge', 'Power drill', 'spare bulb', 'Drains and culverts', 'Glass Table', 'Brick fence', 'humidity meter', 'Clothes hamper', 'air tank', 'Dumbbell', 'phone case', 'Fuse', 'Toaster', 'Polyester', 'Cat trees', 'velvet ribbon', 'lace', 'Handrail', 'GPS device', 'Epoxy', 'Lawn chair', 'northern mockingbird', 'takeout container', 'mountaineer', 'Plastic fork', 'Apple tart', 'peeler', 'Bicycle', 'armoire', 'chimney', 'Headlamp', 'Vacuum sealer', 

## Save as JSONL

In [96]:
with open("20qs-data/final_games/gpt-4o-mini-twice-removed-games.jsonl", "w") as f:
    for i in new_games_won_words:
        json.dump(i, f)
        f.write('\n')

In [59]:
new_games_won = [game for game in game_results if game['win'] == True]

with open("20qs-data/final_games/gpt-4o-mini-games-won.jsonl", "w") as f:
    for i in new_games_won:
        json.dump(i, f)
        f.write('\n')